In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays
import pytz

from datetime import datetime
from scipy import stats

import eli5
from eli5.sklearn import PermutationImportance
from catboost import *
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist

In [33]:
drivers = pd.read_csv('data/drivers.csv')
train = pd.read_csv('data/train_80.csv', parse_dates=['date'])
test = pd.read_csv('data/test.csv', parse_dates=['date'])

anomaly_driver_idx = [320227, 799250, 800600]

In [34]:
train = train.drop('day', axis=1)

In [35]:
drivers2 = drivers[~drivers['driver_id'].isin(anomaly_driver_idx)].reset_index(drop=True)
train2 = train[~train['driver_id'].isin(anomaly_driver_idx)].reset_index(drop=True)
test2 = test[~test['driver_id'].isin(anomaly_driver_idx)].reset_index(drop=True)

In [36]:
date = pd.date_range(start='6/1/2017', end='6/21/2017')
len_driver = len(test2['driver_id'].unique())
len_date = len(date)

df = pd.DataFrame()
df['driver_id'] = np.tile(test2['driver_id'].unique(), len_date)
df['date'] = date.repeat(len_driver)
df = df.sort_values(['driver_id', 'date'])

train_merge = pd.merge(df, train2, on=['driver_id', 'date'], how='left', validate = 'm:1')
train_merge.fillna(0, inplace=True)

train_merge = pd.merge(train_merge, drivers2, on='driver_id', how='left', validate = 'm:1')

In [37]:
test_merge = pd.merge(test2, drivers2, on='driver_id', how='left', validate = 'm:1')

In [38]:
train_merge.head()

,driver_id,date,online_hours,gender,age,number_of_kids
0,111556,2017-06-01,2.216667,FEMALE,49,4
1,111556,2017-06-02,2.500000,FEMALE,49,4
2,111556,2017-06-03,0.000000,FEMALE,49,4
3,111556,2017-06-04,0.000000,FEMALE,49,4
4,111556,2017-06-05,4.666667,FEMALE,49,4


In [39]:
test_merge.head()

,driver_id,date,online_hours,gender,age,number_of_kids
0,979863,2017-06-28,7,MALE,26,2
1,979863,2017-06-27,9,MALE,26,2
2,979863,2017-06-26,9,MALE,26,2
3,979863,2017-06-25,10,MALE,26,2
4,979863,2017-06-24,9,MALE,26,2


# Known Driver 

In [40]:
def additional_features(df):
    #TODO : mean encoding ?
    df['gender'] = (df['gender']=='MALE').astype(int)
    
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    
    df['holiday'] = (df['dayofweek'] > 4).astype(int)
#     national_holiday = (df['day']==1)|(df['day']>25)
#     df.loc[national_holiday, 'holiday'] = 1

    df.sort_values(['driver_id', 'date'], ascending=False, inplace=True)
    df['prev_holiday'] = df['holiday'].shift(1).fillna(0).astype(int)
    df['next_holiday'] = df['holiday'].shift(-1).fillna(0).astype(int)

In [41]:
additional_features(train_merge)
additional_features(test_merge)

In [42]:
test_merge[test_merge.driver_id==979863]

,driver_id,date,online_hours,gender,age,number_of_kids,day,dayofweek,holiday,prev_holiday,next_holiday
0,979863,2017-06-28,7,1,26,2,28,2,0,0,0
1,979863,2017-06-27,9,1,26,2,27,1,0,0,0
2,979863,2017-06-26,9,1,26,2,26,0,0,0,1
3,979863,2017-06-25,10,1,26,2,25,6,1,0,1
4,979863,2017-06-24,9,1,26,2,24,5,1,1,0
5,979863,2017-06-23,8,1,26,2,23,4,0,1,0
6,979863,2017-06-22,7,1,26,2,22,3,0,0,0


In [43]:
train_merge.loc[(train_merge['dayofweek']==6)]

,driver_id,date,online_hours,gender,age,number_of_kids,day,dayofweek,holiday,prev_holiday,next_holiday
52370,998740,2017-06-18,0.000000,1,27,0,18,6,1,0,1
52363,998740,2017-06-11,0.000000,1,27,0,11,6,1,0,1
52356,998740,2017-06-04,0.000000,1,27,0,4,6,1,0,1
52349,998229,2017-06-18,6.433333,1,31,0,18,6,1,0,1
52342,998229,2017-06-11,7.000000,1,31,0,11,6,1,0,1
52335,998229,2017-06-04,6.200000,1,31,0,4,6,1,0,1
52328,998215,2017-06-18,0.000000,0,52,2,18,6,1,0,1
52321,998215,2017-06-11,0.000000,0,52,2,11,6,1,0,1
52314,998215,2017-06-04,0.000000,0,52,2,4,6,1,0,1
52307,997510,2017-06-18,3.833333,1,61,3,18,6,1,0,1


In [44]:
driver_group = train_merge.groupby('driver_id')['online_hours'].agg(['mean', 'std']).reset_index()
driver_group_2 = train_merge.groupby(['driver_id','holiday'])['online_hours'].agg(['mean', 'std']).reset_index()
driver_group_3 = driver_group_2[driver_group_2.holiday ==1]

In [45]:
def additional_features_2(df):    
    df.loc[(df['age']<=40), 'age2'] = 0
    df.loc[(df['age']>40), 'age2'] = 1
    df['age3'] = df['age'] // 10
    
    driver_0 = driver_group[driver_group['mean']<=5]['driver_id'].unique()
    driver_1 = driver_group[driver_group['mean']>5]['driver_id'].unique()
    df['c1'] = 0
    df.loc[df['driver_id'].isin(driver_1), 'c1'] = 1
    
    driver_0 = driver_group[driver_group['mean']<=2]['driver_id'].unique()
    driver_1 = driver_group[(driver_group['mean']>2)&(driver_group['mean']<=4)]['driver_id'].unique()
    driver_2 = driver_group[(driver_group['mean']>4)&(driver_group['mean']<=7)]['driver_id'].unique()
    driver_3 = driver_group[driver_group['mean']>7]['driver_id'].unique()
    df['c2'] = 0
    df.loc[df['driver_id'].isin(driver_1), 'c2'] = 1
    df.loc[df['driver_id'].isin(driver_2), 'c2'] = 2
    df.loc[df['driver_id'].isin(driver_3), 'c2'] = 3
    
    driver_0 = driver_group_3[driver_group_3['mean']<=3]['driver_id'].unique()
    driver_1 = driver_group_3[driver_group_3['mean']>3]['driver_id'].unique()
    df['c3'] = 0
    df.loc[df['driver_id'].isin(driver_1), 'c3'] = 1
    
    df['gender_kids'] = None
    df.loc[(df['gender'] == 0)&(df['number_of_kids'] == 0), ['gender_kids']] = 0
    df.loc[(df['gender'] == 0)&(df['number_of_kids'] == 1), ['gender_kids']] = 1
    df.loc[(df['gender'] == 0)&(df['number_of_kids'] == 2), ['gender_kids']] = 2
    df.loc[(df['gender'] == 0)&(df['number_of_kids'] == 3), ['gender_kids']] = 3
    df.loc[(df['gender'] == 0)&(df['number_of_kids'] == 4), ['gender_kids']] = 4
    df.loc[(df['gender'] == 1)&(df['number_of_kids'] == 0), ['gender_kids']] = 5
    df.loc[(df['gender'] == 1)&(df['number_of_kids'] == 1), ['gender_kids']] = 6
    df.loc[(df['gender'] == 1)&(df['number_of_kids'] == 2), ['gender_kids']] = 7
    df.loc[(df['gender'] == 1)&(df['number_of_kids'] == 3), ['gender_kids']] = 8
    df.loc[(df['gender'] == 1)&(df['number_of_kids'] == 4), ['gender_kids']] = 9
    
    df['age_kids'] = None
    df.loc[(df['age2'] == 0)&(df['number_of_kids'] == 0), ['age_kids']] = 0
    df.loc[(df['age2'] == 0)&(df['number_of_kids'] == 1), ['age_kids']] = 1
    df.loc[(df['age2'] == 0)&(df['number_of_kids'] == 2), ['age_kids']] = 2
    df.loc[(df['age2'] == 0)&(df['number_of_kids'] == 3), ['age_kids']] = 3
    df.loc[(df['age2'] == 0)&(df['number_of_kids'] == 4), ['age_kids']] = 4
    df.loc[(df['age2'] == 1)&(df['number_of_kids'] == 0), ['age_kids']] = 5
    df.loc[(df['age2'] == 1)&(df['number_of_kids'] == 1), ['age_kids']] = 6
    df.loc[(df['age2'] == 1)&(df['number_of_kids'] == 2), ['age_kids']] = 7
    df.loc[(df['age2'] == 1)&(df['number_of_kids'] == 3), ['age_kids']] = 8
    df.loc[(df['age2'] == 1)&(df['number_of_kids'] == 4), ['age_kids']] = 9
    return df

In [46]:
train_merge_2 = additional_features_2(train_merge)
test_merge_2 = additional_features_2(test_merge)

In [47]:
def get_features_3(train, test, prev_i=2):
    start = test.day.min() - 1 - 7*prev_i
    combined = pd.concat([train[train.day>7], test])
    df_train = additional_features_3(train)
    df_test = additional_features_3(combined)
    return df_train, df_test

def additional_features_3(merge):
    df = merge.copy().sort_values(['driver_id', 'date'])
    for i in range(1, 15):
        df['prev_' + str(i)] = df.groupby('driver_id')['online_hours'].shift(i)
        
    df.dropna(inplace=True)
    return df

In [48]:
train_merge_3, test_merge_3 = get_features_3(train_merge_2, test_merge_2)

In [49]:
active_drivers = train_merge_3.groupby(['driver_id']).filter(lambda x: x['online_hours'].mean() > 0.5).driver_id.unique()
lebaran = test_merge_3['day'].isin([25, 26])
off = test_merge_3['online_hours'] == 0
active = test_merge_3['driver_id'].isin(active_drivers)
anomaly_index = (active)&(lebaran)&(off)
np.sum(anomaly_index)

1480

In [50]:
drop_cols = ['driver_id', 'day', 'date', 'online_hours', 'holiday', 'next_holiday', 'prev_holiday']
drop_cols = ['driver_id', 'day', 'date', 'online_hours', 'holiday']
x_train = train_merge_3.drop(drop_cols, axis=1)
y_train = train_merge_3['online_hours']

x_test = test_merge_3.drop(drop_cols, axis=1)
y_test = test_merge_3['online_hours']

In [51]:
train_merge_3.to_csv('data/train_v2.csv', index=False)
test_merge_3.to_csv('data/test_v2.csv', index=False)

In [21]:
# test_merge_3[~anomaly_index].to_csv('data/test_non_anomaly_v2.csv', index=False)

In [22]:
x_train.head(2)

,gender,age,number_of_kids,dayofweek,prev_holiday,next_holiday,age2,age3,c1,c2,...,prev_5,prev_6,prev_7,prev_8,prev_9,prev_10,prev_11,prev_12,prev_13,prev_14
14,0,49,4,3,0,0,1.0,4,0,0,...,0.0,4.433333,3.179167,2.629167,3.116667,4.666667,0.000000,0.0,2.5,2.216667
15,0,49,4,4,0,1,1.0,4,0,0,...,0.0,0.000000,4.433333,3.179167,2.629167,3.116667,4.666667,0.0,0.0,2.500000


# Regressor 

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
def init_model():
    models = {}
    models['ext'] = ExtraTreesRegressor(random_state=0, verbose=0)
    models['ada'] = AdaBoostRegressor(random_state=0)
    models['catboost'] = CatBoostRegressor(random_state=0, verbose=0)
    models['lgbm'] = LGBMRegressor(random_state=0, verbose=0)
    models['xgb'] = XGBRegressor(random_state=0, obj='reg:squarederror')
    models['rf'] = RandomForestRegressor(random_state=0)
    models['dt'] = DecisionTreeRegressor(random_state=0)
    return models

def experiment(x_train, y_train, 
               x_test, y_test, 
               df_test, base_name = ''):
    models = init_model()
    for m in models:
        if m in ['rf', 'dt', 'ext', 'ada']:
            models[m].fit(x_train,y_train)
        else :
            models[m].fit(x_train,y_train, 
                          eval_set=[(x_test, y_test)], 
                          early_stopping_rounds=10, verbose=0)

        if m not in ['dt']:
            importance = models[m].feature_importances_
            for i in np.argsort(importance)[::-1][:3]:
                print(x_train.columns[i], "%.2f"% importance[i])

        y_predicted = models[m].predict(x_test).clip(0, 11)
        rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
        r2 = r2_score(y_test, y_predicted)
        print(m, "================ RMSE: %.5f ===========R2: %.5f"% (rmse, r2))
        print('')
        
        df_test['pred_'+m] = y_predicted
        df_test.to_csv('preds/pred'+base_name+'_v2.csv', index=False)
        # 1.78

In [25]:
df_test = test_merge_3[['driver_id', 'date', 'online_hours']]

In [24]:
experiment(x_train, y_train, x_test, y_test, df_test)

c1 0.28
prev_14 0.22
prev_7 0.17
ext ================ RMSE: 1.91880 ===========R2: 0.67834

prev_14 0.38
prev_7 0.27
c2 0.20
ada ================ RMSE: 2.03972 ===========R2: 0.63652

prev_7 30.82
prev_14 28.53
c2 8.47
catboost ================ RMSE: 1.81412 ===========R2: 0.71248

prev_7 138.00
prev_14 135.00
c2 86.00
lgbm ================ RMSE: 1.81397 ===========R2: 0.71253

[16:30:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.32
c1 0.26
prev_7 0.23
xgb ================ RMSE: 1.83102 ===========R2: 0.70710

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.90563 ===========R2: 0.68274

dt ================ RMSE: 2.61613 ===========R2: 0.40207



In [25]:
experiment(x_train, y_train,
           x_test[~anomaly_index], y_test[~anomaly_index], 
           df_test[~anomaly_index], '_anomaly')

c1 0.28
prev_14 0.22
prev_7 0.17
ext ================ RMSE: 1.89212 ===========R2: 0.65934

prev_14 0.38
prev_7 0.27
c2 0.20
ada ================ RMSE: 2.03852 ===========R2: 0.60458

prev_7 32.98
prev_14 30.43
c2 8.69
catboost ================ RMSE: 1.77947 ===========R2: 0.69869

prev_14 159.00
prev_7 156.00
c2 91.00
lgbm ================ RMSE: 1.77588 ===========R2: 0.69991

[16:31:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.32
c1 0.26
prev_7 0.23
xgb ================ RMSE: 1.79232 ===========R2: 0.69433

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.87764 ===========R2: 0.66453

dt ================ RMSE: 2.63889 ===========R2: 0.33737



# Manually Old Version 

In [26]:
# dt = DecisionTreeRegressor() 
# dt.fit(x_train,y_train)
# y_predicted = dt.predict(x_test).clip(0, 11)
# rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
# print("anomaly RMSE: %.5f"% rmse)
# rmse = np.sqrt(mean_squared_error(y_test[~anomaly_index], 
#                                   y_predicted[~anomaly_index]))
# print("RMSE: %.5f"% rmse)
# # 2.45 without prev_2

In [27]:
# rf = RandomForestRegressor() 
# rf.fit(x_train,y_train)
# y_predicted = rf.predict(x_test).clip(0, 11)
# rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
# print("anomaly RMSE: %.5f"% rmse)
# rmse = np.sqrt(mean_squared_error(y_test[~anomaly_index], 
#                                   y_predicted[~anomaly_index]))
# print("RMSE: %.5f"% rmse)
# # 1.94

In [28]:
# {x_train.columns[i]:rf.feature_importances_[i] for i in np.argsort(rf.feature_importances_)}

In [29]:
# xgb = XGBRegressor()
# xgb.fit(x_train,y_train)
# y_predicted = xgb.predict(x_test).clip(0, 11)
# rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
# print("anomaly RMSE: %.5f"% rmse)
# rmse = np.sqrt(mean_squared_error(y_test[~anomaly_index], 
#                                   y_predicted[~anomaly_index]))
# print("RMSE: %.5f"% rmse)
# # 1.8

In [30]:
# {x_train.columns[i]:xgb.feature_importances_[i] for i in np.argsort(xgb.feature_importances_)}

In [31]:
# lgbm = LGBMRegressor()
# lgbm.fit(x_train,y_train)
# y_predicted = lgbm.predict(x_test).clip(0, 11)
# rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
# print("anomaly RMSE: %.5f"% rmse)
# rmse = np.sqrt(mean_squared_error(y_test[~anomaly_index], 
#                                   y_predicted[~anomaly_index]))
# print("RMSE: %.5f"% rmse)
# # 1.78

In [32]:
# {x_train.columns[i]:lgbm.feature_importances_[i] for i in np.argsort(lgbm.feature_importances_)}

In [33]:
# cb = CatBoostRegressor(verbose=0)
# cb.fit(x_train,y_train)
# y_predicted = cb.predict(x_test).clip(0, 11)
# rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
# print("anomaly RMSE: %.5f"% rmse)
# rmse = np.sqrt(mean_squared_error(y_test[~anomaly_index], 
#                                   y_predicted[~anomaly_index]))
# print("RMSE: %.5f"% rmse)
# # 1.79

In [34]:
# {x_train.columns[i]:cb.feature_importances_[i] for i in np.argsort(cb.feature_importances_)}

# Cluster 

In [26]:
train_cluster = train_merge_3.copy()

In [27]:
n_cluster = [3, 5, 10]
vectors = train_cluster[['prev_'+ str(i) for i in range(1, 8)]].values
kmeans = {c:KMeans(n_clusters=c, random_state=0).fit(vectors) for c in n_cluster}

for c in n_cluster:
    train_cluster['cr_'+str(c)] = kmeans[c].predict(vectors)
    
cluster_group = train_cluster[['driver_id', 'dayofweek']+['cr_'+str(c) for c in n_cluster]]
train_merge_cluster = pd.merge(train_merge_3, cluster_group, on=['driver_id', 'dayofweek'], how='left')
test_merge_cluster = pd.merge(test_merge_3, cluster_group, on=['driver_id', 'dayofweek'], how='left')
test_merge_cluster.isnull().any().any(), train_merge_cluster.isnull().any().any()

(False, False)

In [28]:
x_train_0 = train_merge_cluster.drop(drop_cols, axis=1)
y_train_0 = train_merge_cluster['online_hours']

x_test_0 = test_merge_cluster.drop(drop_cols, axis=1)
y_test_0 = test_merge_cluster['online_hours']

In [30]:
experiment(x_train_0, y_train_0, x_test_0, y_test_0, df_test, '_cluster')

prev_14 0.27
c1 0.21
prev_7 0.16
ext ================ RMSE: 1.92893 ===========R2: 0.67494



FileNotFoundError: [Errno 2] No such file or directory: 'preds/pred_cluster_v2.csv'

In [ ]:
experiment(x_train_0, y_train_0, 
           x_test_0[~anomaly_index], y_test_0[~anomaly_index], 
           df_test[~anomaly_index], '_cluster_no_anomaly')

In [29]:
train_merge_cluster.to_csv('data/train_cluster_v2.csv', index=False)
test_merge_cluster.to_csv('data/test_cluster_v2.csv', index=False)
test_merge_cluster[~anomaly_index].to_csv('data/test_cluster_v2.csv', index=False)

In [39]:
lgbm = LGBMRegressor()
lgbm = lgbm.fit(x_train, y_train)
perm = PermutationImportance(lgbm).fit(x_test, y_test)
eli5.show_weights(perm)

Weight,Feature
0.3806 ± 0.0095,x19
0.3553 ± 0.0056,x26
0.0912 ± 0.0040,x9
0.0493 ± 0.0038,x8
0.0101 ± 0.0008,x1
0.0089 ± 0.0009,x3
0.0034 ± 0.0008,x11
0.0030 ± 0.0008,x13
0.0029 ± 0.0012,x10
0.0022 ± 0.0006,x20


In [40]:
negative_features_2 = x_train.columns[perm.feature_importances_<=0]
negative_features_2

Index(['next_holiday', 'age2', 'age3', 'age_kids'], dtype='object')

In [41]:
{x_train.columns[i]:perm.feature_importances_[i] for i in np.argsort(perm.feature_importances_)}

{'next_holiday': -9.275326363125113e-05,
 'age_kids': -5.175377183392893e-05,
 'age2': 0.0,
 'age3': 0.0,
 'prev_3': 3.279493719472448e-05,
 'prev_holiday': 4.9890341127079016e-05,
 'number_of_kids': 5.512975587897984e-05,
 'prev_5': 0.0005381479694727797,
 'prev_4': 0.0006527233151575995,
 'prev_11': 0.0007986063359915585,
 'prev_9': 0.0010350170033242678,
 'prev_12': 0.0010561432697484773,
 'prev_6': 0.001162602071911345,
 'prev_2': 0.0012788567036071895,
 'prev_10': 0.0013330917602947024,
 'gender': 0.0017115363738182455,
 'prev_13': 0.0021846368592141506,
 'prev_8': 0.0021947394070158444,
 'c3': 0.002888888134304568,
 'prev_1': 0.002999575824903733,
 'gender_kids': 0.0034225562577359534,
 'dayofweek': 0.008863653723759768,
 'age': 0.01008659360902906,
 'c1': 0.049264624383015064,
 'c2': 0.09117426594160039,
 'prev_14': 0.3553381976134104,
 'prev_7': 0.38055948515368404}

In [58]:
x_train_2 = train_merge_cluster.drop(drop_cols, axis=1)
x_train_2 = x_train_2.drop(negative_features_2, axis=1)
y_train_2 = train_merge_cluster['online_hours']

x_test_2 = test_merge_cluster.drop(drop_cols, axis=1)
x_test_2 = x_test_2.drop(negative_features_2, axis=1)
y_test_2 = test_merge_cluster['online_hours']

In [59]:
experiment(x_train_2, y_train_2, x_test_2, y_test_2, df_test, '_perm')

prev_14 0.25
prev_7 0.23
c1 0.21
ext ================ RMSE: 1.92464 ===========R2: 0.67638

prev_14 0.41
prev_7 0.29
c2 0.17
ada ================ RMSE: 2.04742 ===========R2: 0.63378

prev_7 32.50
prev_14 29.58
c2 8.49
catboost ================ RMSE: 1.81516 ===========R2: 0.71215

prev_7 148.00
prev_14 140.00
c2 86.00
lgbm ================ RMSE: 1.81122 ===========R2: 0.71340

[16:37:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.31
c1 0.28
prev_7 0.23
xgb ================ RMSE: 1.82944 ===========R2: 0.70761

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.92129 ===========R2: 0.67751

dt ================ RMSE: 2.66178 ===========R2: 0.38102



In [60]:
experiment(x_train_2, y_train_2, 
           x_test_2[~anomaly_index], y_test_2[~anomaly_index], 
           df_test[~anomaly_index], '_perm_no_anomaly')

prev_14 0.25
prev_7 0.23
c1 0.21
ext ================ RMSE: 1.95255 ===========R2: 0.65376

prev_14 0.41
prev_7 0.29
c2 0.17
ada ================ RMSE: 2.06402 ===========R2: 0.61310

prev_7 32.50
prev_14 29.58
c2 8.49
catboost ================ RMSE: 1.84088 ===========R2: 0.69223

prev_7 148.00
prev_14 140.00
c2 86.00
lgbm ================ RMSE: 1.83839 ===========R2: 0.69307

[16:37:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.31
c1 0.28
prev_7 0.23
xgb ================ RMSE: 1.85498 ===========R2: 0.68750

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.95024 ===========R2: 0.65458

dt ================ RMSE: 2.69956 ===========R2: 0.33816



In [61]:
lgbm = LGBMRegressor()
selector = RFE(lgbm, len(x_train.columns)-len(negative_features_2), step=1)
selector = selector.fit(x_train, y_train)
negative_features_1 = x_train.columns[~selector.support_]
negative_features_1

Index(['gender', 'number_of_kids', 'age2', 'age3', 'age_kids'], dtype='object')

In [62]:
x_train_1 = train_merge_cluster.drop(drop_cols, axis=1)
x_train_1 = x_train_1.drop(negative_features_1, axis=1)
y_train_1 = train_merge_cluster['online_hours']

x_test_1 = test_merge_cluster.drop(drop_cols, axis=1)
x_test_1 = x_test_1.drop(negative_features_1, axis=1)
y_test_1 = test_merge_cluster['online_hours']

In [63]:
experiment(x_train_1, y_train_1, x_test_1, y_test_1, df_test, '_recursive')

prev_14 0.24
prev_7 0.22
c1 0.21
ext ================ RMSE: 1.92086 ===========R2: 0.67765

prev_14 0.38
prev_7 0.27
c2 0.20
ada ================ RMSE: 2.03972 ===========R2: 0.63652

prev_7 32.32
prev_14 30.24
c2 7.77
catboost ================ RMSE: 1.81601 ===========R2: 0.71188

prev_7 162.00
prev_14 140.00
c2 82.00
lgbm ================ RMSE: 1.81166 ===========R2: 0.71326

[16:37:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.33
prev_7 0.25
c1 0.24
xgb ================ RMSE: 1.83065 ===========R2: 0.70722

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.92187 ===========R2: 0.67731

dt ================ RMSE: 2.67105 ===========R2: 0.37670



In [64]:
experiment(x_train_1, y_train_1, 
           x_test_1[~anomaly_index], y_test_1[~anomaly_index], 
           df_test[~anomaly_index], '_recursive_no_anomaly')

prev_14 0.24
prev_7 0.22
c1 0.21
ext ================ RMSE: 1.94953 ===========R2: 0.65483

prev_14 0.38
prev_7 0.27
c2 0.20
ada ================ RMSE: 2.05614 ===========R2: 0.61605

prev_7 32.32
prev_14 30.24
c2 7.77
catboost ================ RMSE: 1.84157 ===========R2: 0.69201

prev_7 167.00
prev_14 143.00
c2 83.00
lgbm ================ RMSE: 1.83849 ===========R2: 0.69303

[16:38:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
prev_14 0.33
prev_7 0.25
c1 0.24
xgb ================ RMSE: 1.85598 ===========R2: 0.68717

prev_14 0.45
prev_7 0.24
c2 0.03
rf ================ RMSE: 1.94928 ===========R2: 0.65492

dt ================ RMSE: 2.70315 ===========R2: 0.33640

